## Import Library

In [12]:
import numpy as np

import neuralnetwork

from neuralnetwork import layers
from neuralnetwork import activations

## Development Testing

In [13]:
from keras import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(10, input_shape=(50, 1), return_sequences=False))
model.add(Dense(1, activation='linear'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10)                480       
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________


In [14]:
a = np.array([[1, 2]])

a.transpose()

array([[1],
       [2]])

## Test Model Summary

In [15]:
model = neuralnetwork.Sequential()

model.add(layers.LSTM(10, input_shape=(50, 1)))
model.add(layers.Dense(1, activation=activations.Softmax))

model.summary()
print()


Model: Sequential
Layer (type)                       Output Shape                       Param #       
lstm_1 (LSTM)                      (None, 10)                         480              
--------------------------------------------------------------------------------
dense_1 (Dense)                    (None, 1)                          11               
Total params: 491




## Test LSTM

### IF4071 Slide Calculation Example

### LSTM Forward Propagation with Random Weights

### LSTM Forward Propagation with Keras Trained Model Weights

In [16]:
import pandas as pd

pd.options.mode.chained_assignment = None

training_data   = pd.read_csv("dataset/bitcoin/training.csv")
test_data       = pd.read_csv("dataset/bitcoin/test.csv")

In [17]:
training_data = training_data[::-1].reset_index(drop=True)
test_data = test_data[::-1].reset_index(drop=True)


In [18]:
training_data = training_data.drop(columns=['Date'])
test_data = test_data.drop(columns=['Date'])

training_data

,Open,High,Low,Close,Volume,Market Cap
0,135.30,135.98,132.10,134.21,-,"1,500,520,000"
1,134.44,147.49,134.00,144.54,-,"1,491,160,000"
2,144.00,146.93,134.05,139.00,-,"1,597,780,000"
3,139.00,139.89,107.72,116.99,-,"1,542,820,000"
4,116.38,125.60,92.28,105.21,-,"1,292,190,000"
...,...,...,...,...,...,...
1551,2538.71,2693.32,2529.34,2671.78,"789,104,000","41,816,500,000"
1552,2679.73,2897.45,2679.73,2809.01,"1,380,100,000","44,144,400,000"
1553,2807.02,2808.76,2692.80,2726.45,"803,746,000","46,246,700,000"
1554,2724.39,2758.53,2644.85,2757.18,"705,943,000","44,890,700,000"


In [19]:
from sklearn.model_selection import train_test_split

temp = training_data[training_data['Volume'] != '-']
temp['Volume']  = temp['Volume'].apply(lambda x: float(x.replace(",","")))
temp['Market Cap']  = temp['Market Cap'].apply(lambda x: float(x.replace(",","")))

need_fill = training_data[training_data['Volume'] == '-']

need_fill = need_fill.loc[:, ['Open', 'Close', 'Low', 'High', 'Market Cap']]
need_fill['Market Cap']  = need_fill['Market Cap'].apply(lambda x: float(x.replace(",","")))
need_fill_index = need_fill.index.to_list()

X_temp = temp.loc[:, ['Open', 'Close', 'Low', 'High', 'Market Cap']]
y_temp = temp.loc[:, ['Volume']]

filled = training_data[training_data['Volume'] != '-']
filled_index = filled.index.to_list()

X_temp_train, X_temp_test, y_temp_train, y_temp_test = train_test_split(X_temp, y_temp, test_size=0.1, random_state=20)

temp

,Open,High,Low,Close,Volume,Market Cap
243,763.28,777.51,713.60,735.07,4.686270e+07,9.295570e+09
244,737.98,747.06,705.35,727.83,3.250580e+07,8.990850e+09
245,728.05,748.61,714.44,745.05,1.901130e+07,8.872600e+09
246,741.35,766.60,740.24,756.13,2.070770e+07,9.037000e+09
247,760.32,760.58,738.17,754.01,2.089730e+07,9.268240e+09
...,...,...,...,...,...,...
1551,2538.71,2693.32,2529.34,2671.78,7.891040e+08,4.181650e+10
1552,2679.73,2897.45,2679.73,2809.01,1.380100e+09,4.414440e+10
1553,2807.02,2808.76,2692.80,2726.45,8.037460e+08,4.624670e+10
1554,2724.39,2758.53,2644.85,2757.18,7.059430e+08,4.489070e+10


In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score

reg = LinearRegression().fit(X_temp, y_temp)

y_pred = reg.predict(X_temp_test)

explained_variance_score(y_temp_test, y_pred)

0.9012728856978025

In [21]:
need_fill_pred = reg.predict(need_fill)

training_data['Market Cap']  = training_data['Market Cap'].apply(lambda x: float(x.replace(",","")))
training_data.loc[need_fill_index, ['Volume']] = need_fill_pred

training_data.loc[filled_index, ['Volume']] = list(map(lambda x: float(x[0].replace(',', '')), training_data.loc[filled_index, ['Volume']].to_numpy()))

In [22]:
X_train = training_data

X_train

,Open,High,Low,Close,Volume,Market Cap
0,135.30,135.98,132.10,134.21,-77722615.178571,1.500520e+09
1,134.44,147.49,134.00,144.54,-45834207.602472,1.491160e+09
2,144.00,146.93,134.05,139.00,-55816196.068126,1.597780e+09
3,139.00,139.89,107.72,116.99,-11651543.760081,1.542820e+09
4,116.38,125.60,92.28,105.21,-1315126.751554,1.292190e+09
...,...,...,...,...,...,...
1551,2538.71,2693.32,2529.34,2671.78,789104000.0,4.181650e+10
1552,2679.73,2897.45,2679.73,2809.01,1380100000.0,4.414440e+10
1553,2807.02,2808.76,2692.80,2726.45,803746000.0,4.624670e+10
1554,2724.39,2758.53,2644.85,2757.18,705943000.0,4.489070e+10


In [23]:
X_test = test_data
X_test['Volume']  = X_test['Volume'].apply(lambda x: float(x.replace(",","")))
X_test['Market Cap']  = X_test['Market Cap'].apply(lambda x: float(x.replace(",","")))

X_test

,Open,High,Low,Close,Volume,Market Cap
0,2871.30,2921.35,2685.61,2718.26,1.324670e+09,4.732180e+10
1,2727.13,2762.53,2668.59,2710.67,1.094950e+09,4.495080e+10
2,2709.56,2813.31,2685.14,2804.73,8.047970e+08,4.466640e+10
3,2806.93,2899.33,2743.72,2895.89,1.002120e+09,4.627620e+10
4,2897.63,3290.01,2874.83,3252.91,1.945700e+09,4.777820e+10
5,3257.61,3293.29,3155.60,3213.94,1.105030e+09,5.372090e+10
6,3212.78,3397.68,3180.89,3378.94,1.482280e+09,5.298730e+10


In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'])
X_test = pd.DataFrame(scaler.transform(X_test), columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'])

In [25]:
X_train[:8]

,Open,High,Low,Close,Volume,Market Cap
0,0.023156,0.020996,0.023989,0.022764,0.001727,0.015149
1,0.022858,0.024930,0.024674,0.026339,0.013752,0.014952
2,0.026172,0.024739,0.024692,0.024421,0.009988,0.017192
3,0.024439,0.022332,0.015204,0.016805,0.026642,0.016037
4,0.016598,0.017447,0.009640,0.012728,0.030540,0.010773
5,0.013086,0.011476,0.004890,0.010146,0.026824,0.008418
6,0.010261,0.013824,0.009719,0.015251,0.024804,0.006524
7,0.015391,0.015123,0.014995,0.016431,0.011156,0.009987


In [26]:
X_test

,Open,High,Low,Close,Volume,Market Cap
0,0.971602,0.973145,0.944173,0.916998,0.530566,0.977539
1,0.921625,0.918854,0.938040,0.914371,0.443939,0.927741
2,0.915534,0.936213,0.944004,0.946921,0.334523,0.921768
3,0.949288,0.965618,0.965114,0.978468,0.408933,0.955578
4,0.980729,1.099168,1.012360,1.102018,0.764755,0.987125
5,1.105518,1.100289,1.113539,1.088532,0.447740,1.111940
6,1.089978,1.135973,1.122652,1.145632,0.590000,1.096532


In [63]:
timeseries = X_test['Open'].to_numpy()

def create_trainer(timeseries, steps):
    X = []
    Y = []
    for i in range(steps-1, len(timeseries)-1):
        re = []
        for j in range(i-steps+1, i+1):
            re.append(timeseries[j])
        Y.append(timeseries[i+1])
        X.append(re)
    return np.array(X), np.array(Y)


def create_trainer_extended(timeseries, steps):
    X = []
    Y = []
    for i in range(steps-1, len(timeseries)-1):
        re = []
        for j in range(i-steps+1, i+1):
            r = []
            for k in range(len(timeseries[j])):
                r.append(timeseries[j][k])
            re.append(r)
        Y.append(timeseries[i+1])
        X.append(re)
    return np.array(X), np.array(Y)

X_, Y_ = create_trainer(timeseries, 6)

X, Y = create_trainer(X_train['Market Cap'].to_numpy(), 32)
print(X.shape)

X = X.reshape([1524, 1, 32])


print(Y.shape)

(1524, 32)
(1524,)


In [66]:
timeseries = X_train.to_numpy()

print(timeseries.shape)

X, Y = create_trainer_extended(timeseries, 6)

print(X.shape)

print(X)

(1556, 6)
(1550, 6, 6)
[[[0.02315649 0.02099578 0.02398919 0.02276377 0.00172674 0.01514882]
  [0.02285837 0.02493035 0.02467387 0.02633856 0.01375179 0.01495224]
  [0.02617238 0.02473892 0.02469189 0.02442139 0.00998761 0.01719159]
  [0.02443911 0.02233237 0.0152036  0.01680463 0.026642   0.01603726]
  [0.0165978  0.01744748 0.00963964 0.01272805 0.03053984 0.01077324]
  [0.01308619 0.01147555 0.00489009 0.01014645 0.02682445 0.00841837]]

 [[0.02285837 0.02493035 0.02467387 0.02633856 0.01375179 0.01495224]
  [0.02617238 0.02473892 0.02469189 0.02442139 0.00998761 0.01719159]
  [0.02443911 0.02233237 0.0152036  0.01680463 0.026642   0.01603726]
  [0.0165978  0.01744748 0.00963964 0.01272805 0.03053984 0.01077324]
  [0.01308619 0.01147555 0.00489009 0.01014645 0.02682445 0.00841837]
  [0.01026096 0.01382399 0.00971892 0.01525082 0.02480446 0.00652431]]

 [[0.02617238 0.02473892 0.02469189 0.02442139 0.00998761 0.01719159]
  [0.02443911 0.02233237 0.0152036  0.01680463 0.026642   0.016

In [69]:
timeseries = X_test.to_numpy()

X_, Y_ = create_trainer_extended(timeseries, 6)


print(X_.shape)

(1, 6, 6)


In [45]:
X_train

,Open,High,Low,Close,Volume,Market Cap
0,0.023156,0.020996,0.023989,0.022764,0.001727,0.015149
1,0.022858,0.024930,0.024674,0.026339,0.013752,0.014952
2,0.026172,0.024739,0.024692,0.024421,0.009988,0.017192
3,0.024439,0.022332,0.015204,0.016805,0.026642,0.016037
4,0.016598,0.017447,0.009640,0.012728,0.030540,0.010773
...,...,...,...,...,...,...
1551,0.856308,0.895195,0.887859,0.900913,0.328605,0.861911
1552,0.905194,0.964975,0.942054,0.948403,0.551469,0.910804
1553,0.949319,0.934657,0.946764,0.919832,0.334127,0.954959
1554,0.920675,0.917487,0.929485,0.930466,0.297245,0.926479


In [47]:
timeseries = X_train

print(timeseries.shape)

timeseries = X_train['Open']

print(timeseries.shape)

X, Y = create_trainer(timeseries, 6)

print(X.shape)

X = X.reshape([len(X), 6, 1])

print(X)

(1556, 6)
(1556,)
(1550, 6)
[[[0.02315649]
  [0.02285837]
  [0.02617238]
  [0.02443911]
  [0.0165978 ]
  [0.01308619]]

 [[0.02285837]
  [0.02617238]
  [0.02443911]
  [0.0165978 ]
  [0.01308619]
  [0.01026096]]

 [[0.02617238]
  [0.02443911]
  [0.0165978 ]
  [0.01308619]
  [0.01026096]
  [0.01539144]]

 ...

 [[0.94969356]
  [0.92355584]
  [0.93215286]
  [0.86984872]
  [0.85630841]
  [0.90519357]]

 [[0.92355584]
  [0.93215286]
  [0.86984872]
  [0.85630841]
  [0.90519357]
  [0.94931917]]

 [[0.93215286]
  [0.86984872]
  [0.85630841]
  [0.90519357]
  [0.94931917]
  [0.92067514]]]


In [77]:
from keras import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(50, input_shape=(6, 6)))
model.add(Dense(6, activation='linear'))

model.compile(loss='mae', optimizer='adam')

model.fit(X, Y, epochs=50)

Epoch 1/50
49/49 [==============================] - 2s 4ms/step - loss: 0.0564
Epoch 2/50
49/49 [==============================] - 0s 3ms/step - loss: 0.0142
Epoch 3/50
49/49 [==============================] - 0s 4ms/step - loss: 0.0120
Epoch 4/50
49/49 [==============================] - 0s 3ms/step - loss: 0.0113
Epoch 5/50
49/49 [==============================] - 0s 3ms/step - loss: 0.0110
Epoch 6/50
49/49 [==============================] - 0s 3ms/step - loss: 0.0104
Epoch 7/50
49/49 [==============================] - 0s 3ms/step - loss: 0.0103
Epoch 8/50
49/49 [==============================] - 0s 3ms/step - loss: 0.0098
Epoch 9/50
49/49 [==============================] - 0s 3ms/step - loss: 0.0098
Epoch 10/50
49/49 [==============================] - 0s 3ms/step - loss: 0.0095
Epoch 11/50
49/49 [==============================] - 0s 3ms/step - loss: 0.0096
Epoch 12/50
49/49 [==============================] - 0s 3ms/step - loss: 0.0090
Epoch 13/50
49/49 [==============================

In [80]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 50)                11400     
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 306       
Total params: 11,706
Trainable params: 11,706
Non-trainable params: 0
_________________________________________________________________


In [78]:
model.predict(X_)


array([[1.1093804 , 1.0923008 , 1.092881  , 1.0708021 , 0.46934748,
        1.0917736 ]], dtype=float32)

In [79]:
X_test

,Open,High,Low,Close,Volume,Market Cap
0,0.971602,0.973145,0.944173,0.916998,0.530566,0.977539
1,0.921625,0.918854,0.938040,0.914371,0.443939,0.927741
2,0.915534,0.936213,0.944004,0.946921,0.334523,0.921768
3,0.949288,0.965618,0.965114,0.978468,0.408933,0.955578
4,0.980729,1.099168,1.012360,1.102018,0.764755,0.987125
5,1.105518,1.100289,1.113539,1.088532,0.447740,1.111940
6,1.089978,1.135973,1.122652,1.145632,0.590000,1.096532


In [ ]:
units = int(int(model.layers[0].trainable_weights[0].shape[1])/4)
print("No units: ", units)

W = model.layers[0].get_weights()[0]
U = model.layers[0].get_weights()[1]
b = model.layers[0].get_weights()[2]

W_i = W[:, :units]
W_f = W[:, units: units * 2]
W_c = W[:, units * 2: units * 3]
W_o = W[:, units * 3:]

print()
print(W_i.shape)
print(W_f.shape)
print(W_c.shape)
print(W_o.shape)

U_i = U[:, :units]
U_f = U[:, units: units * 2]
U_c = U[:, units * 2: units * 3]
U_o = U[:, units * 3:]

print()
print(U_i.shape)
print(U_f.shape)
print(U_c.shape)
print(U_o.shape)

b_i = b[:units]
b_f = b[units: units * 2]
b_c = b[units * 2: units * 3]
b_o = b[units * 3:]

No units:  50

(6, 50)
(6, 50)
(6, 50)
(6, 50)

(50, 50)
(50, 50)
(50, 50)
(50, 50)


In [ ]:
model.layers[0].get_weights()[0]

array([[-0.1392791 ,  0.05002498, -0.06446438, ...,  0.00967645,
        -0.20632012,  0.01415829],
       [-0.15984777, -0.04183017, -0.17885262, ..., -0.10814803,
         0.00797641, -0.22194402],
       [ 0.15051372, -0.13753623,  0.08001998, ..., -0.15487905,
        -0.0296196 , -0.10843782],
       [-0.03551413, -0.05436983, -0.00073438, ...,  0.08909791,
         0.04144162, -0.07109155],
       [ 0.00607169,  0.07814564,  0.21230817, ...,  0.09079995,
        -0.05582762, -0.02090334],
       [ 0.036235  ,  0.05632178,  0.02760142, ..., -0.00670688,
         0.21089181, -0.10377761]], dtype=float32)

In [ ]:
model.layers[0].get_weights()[1]

array([[ 0.13279188, -0.10687534,  0.06033575, ...,  0.02696105,
         0.00903529, -0.01891052],
       [-0.00402407,  0.03420198,  0.05629672, ...,  0.02683519,
        -0.00775447,  0.117441  ],
       [-0.0291966 , -0.00639475,  0.00279927, ...,  0.04555694,
         0.0028108 , -0.01797051],
       ...,
       [-0.02162166, -0.09385679,  0.00306784, ...,  0.07118312,
         0.0214299 , -0.1209573 ],
       [ 0.12777153,  0.07885771, -0.1552414 , ...,  0.06771434,
         0.01547261,  0.01160152],
       [ 0.1123812 ,  0.10676419,  0.04474918, ..., -0.00696415,
         0.01167692, -0.04706453]], dtype=float32)

In [ ]:
model.layers[1].get_weights()[0]

array([[-1.7379874e-01],
       [-9.6271291e-02],
       [-5.8904476e-02],
       [-2.5841427e-01],
       [-1.7828824e-01],
       [-3.0865955e-01],
       [ 5.9684213e-02],
       [-1.9371882e-01],
       [ 8.0576502e-02],
       [ 3.4744489e-01],
       [-3.2692641e-01],
       [ 2.1849556e-01],
       [-3.4598944e-01],
       [-1.5949889e-01],
       [-1.1862998e-01],
       [ 6.0677800e-02],
       [ 3.4316462e-01],
       [-1.5300912e-01],
       [-2.9334560e-01],
       [-4.0356371e-01],
       [-5.3331636e-02],
       [-7.4939974e-02],
       [ 1.6332662e-01],
       [-3.0379409e-01],
       [-3.3086157e-01],
       [ 1.6624558e-01],
       [ 4.9469292e-01],
       [-2.3757258e-01],
       [ 1.7310390e-03],
       [-7.1009256e-02],
       [-8.3089872e-06],
       [-4.9332969e-02],
       [-1.5700921e-01],
       [ 2.8510493e-01],
       [-3.6026917e-02],
       [-1.6479645e-03],
       [-8.8507198e-02],
       [-2.3703752e-01],
       [-3.7224933e-03],
       [-1.5313992e-01],


In [ ]:
model.layers[1].get_weights()[1]

array([0.0025546], dtype=float32)